# BBNN-QC

Neural Network with time as input and controls as output

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from qutip import qutrit_ops

In [2]:
class Sin(nn.Module):
    """The sin activation function.
    """

    def __init__(self):
        """Initializer method.
        """
        super().__init__()

    def forward(self, input_):
        return torch.sin(input_)

* Define Neural Network

In [3]:
class NeuralNet(nn.Module):
    def __init__(self, hidden_size, nb_hidden_layers = 2, input_size = 1, output_size = 1, activation_fn = nn.Tanh, max_control = 1.):
        super(NeuralNet, self).__init__()

        self.nb_hidden_layers = nb_hidden_layers
        self.hidden_layers = []
        self.hidden_act_layers = []
        self.max_control = max_control

        # input layer
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.relu_input = activation_fn()

        # hidden layers
        for layer in range(nb_hidden_layers):
            new_layer = nn.Linear(hidden_size, hidden_size)
            self.hidden_layers.append(new_layer)
            self.hidden_act_layers.append(activation_fn())

            # hidden layer parameters should be registered
            self.register_parameter(f"weights_{layer}", new_layer.weight)
            self.register_parameter(f"bias_{layer}", new_layer.bias)

        # output layer
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.input_layer(x)
        out = self.relu_input(out)

        for layer in range(self.nb_hidden_layers):
            out = self.hidden_layers[layer](out)
            out = self.hidden_act_layers[layer](out)

        out = self.output_layer(out)
        out = torch.clip(input=out, min=-self.max_control, max=self.max_control)

        return out

* Loss function

In [4]:
criterion = nn.MSELoss()

* Define discretized time, initialize Neural net and Adam optimizer

In [5]:
timings = [(2.4e-3, 2.4, 1e3, 1.), (1.4e-3, 1.4, 1e3, 2.), (1.2e-3, 1.2, 1e3, 3.), (1.1e-3, 1.1, 1e3, 4.)]
print_interval = 100
ξ = 1

In [6]:
def get_concurrence(state):
    c1 = state[0]
    c2 = state[1]
    c3 = state[2]

    return torch.abs(2* c1 * c3 - c2**2)

* Define the objective function, it evolves quantum system in time based on current NN outputs and returns the final fidelity of the terminal state

In [7]:
def criterion_fidelity_custom(u_pred, time, Dt):
    [proj1, _, proj3, trans12, trans23, _] = qutrit_ops()
    trans21 = trans12.dag()
    trans32 = trans23.dag()

    Ω_pred = u_pred[:, 0:1]
    Δ_pred = u_pred[:, 1:2]

    time_tensor = torch.from_numpy(time).reshape(len(time), 1)

    current_state = torch.from_numpy(np.array([1. + 0j, 0, 0]))
    final_state = torch.from_numpy(np.array([0. + 0j, 1., 0]))
    
    H1 = (trans12 + trans21 + trans23 + trans32)

    for i in range(len(time_tensor)):
        H = torch.from_numpy(H1.full()) * Ω_pred[i] / np.sqrt(2)
        H = H + torch.from_numpy(proj3.full()) * (4 * ξ - Δ_pred[i])
        H = H + torch.from_numpy(proj1.full()) * (Δ_pred[i])
        
        current_state = torch.matmul(torch.linalg.matrix_exp(-1j * H * Dt), current_state)

    infidelity = 1 - abs(torch.inner(current_state, final_state)) ** 2

    return infidelity

* Define function to get the quantum state at all intermediate time steps using the optimal controls discovered by the NN after the training

In [8]:
def get_states(u_pred, time, Dt):
    [proj1, _, proj3, trans12, trans23, _] = qutrit_ops()
    trans21 = trans12.dag()
    trans32 = trans23.dag()

    Ω_pred = u_pred[:, 0:1]
    Δ_pred = u_pred[:, 1:2]

    time_tensor = torch.from_numpy(time).reshape(len(time), 1)

    states = []

    current_state = torch.from_numpy(np.array([1. + 0j, 0, 0]))
    final_state = torch.from_numpy(np.array([0. + 0j, 1., 0]))
    
    H1 = (trans12 + trans21 + trans23 + trans32)

    for i in range(len(time_tensor)):
        H = torch.from_numpy(H1.full()) * Ω_pred[i] / np.sqrt(2)
        H = H + torch.from_numpy(proj3.full()) * (4 * ξ - Δ_pred[i])
        H = H + torch.from_numpy(proj1.full()) * (Δ_pred[i])
        
        current_state = torch.matmul(torch.linalg.matrix_exp(-1j * H * Dt), current_state)
        states.append(current_state)

    infidelity = 1 - abs(torch.inner(current_state, final_state)) ** 2

    return states

* Training loop of the Deep NN using the defined objective function

In [ ]:
timing_losses = []
fidelities_array = []
populations_array = []
controls_array = []
timings_array = []
max_control_array = []
final_states_array = []

torch.manual_seed(986532)

In [ ]:
for (Dt, T, N, max_control) in timings:
    max_control_array.append(max_control)
    losses = []
    loss_threshold = 1e-4
    loss_float = 1
    iterations = 10000

    time = np.arange(0, T + Dt, Dt, dtype = np.float32)
    time_tensor = torch.from_numpy(time).reshape(len(time), 1)
    time_tensor.requires_grad_(True)

    timings_array.append(time)

    # create model
    model = NeuralNet(hidden_size=75, input_size=1, output_size=2, nb_hidden_layers=4, activation_fn=nn.Tanh, max_control=max_control)

    learning_rate = 1e-3
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    num_epochs = 0

    model.train()

    # while loss_float >= loss_threshold:
    for iter in range(iterations):
        t_train = time_tensor

        # forward pass
        u_pred = model(t_train)

        # calculate loss based on controls that produced by the nn
        loss = criterion_fidelity_custom(u_pred, time, Dt)
        losses.append(loss.detach().numpy())

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if num_epochs % print_interval == 0:
            print("Epoch = ", num_epochs, ", Infidelity = ", loss.clone().detach().numpy())

        num_epochs += 1

        if losses[-1] < loss_threshold:
            break

    timing_losses.append(losses)

    with torch.no_grad():
        u_pred = model(t_train)

        Ω_pred = u_pred[:, 0:1]
        Δ_pred = u_pred[:, 1:2]

        # transform tensors to arrays
        omegas = np.array([Ω[0] for Ω in Ω_pred.detach().numpy()])
        detunings = np.array([Δ[0] for Δ in Δ_pred.detach().numpy()])

        controls_array.append((omegas, detunings))

        states = get_states(u_pred, time, Dt)

        population1 = [(abs(state[0])**2).item() for state in states]
        population2 = [(abs(state[1])**2).item() for state in states]
        population3 = [(abs(state[2])**2).item() for state in states]

        target_state = torch.from_numpy(np.array([0. + 0j, 1., 0]))
        fidelities = [abs(torch.inner(state, target_state)) ** 2 for state in states]
        final_state = states[-1]
        final_states_array.append(final_state)

        fidelities_array.append(fidelities)
        populations_array.append((population1, population2, population3))

* Plot the controls, the populations, the fidelity and the loss function calues during training

In [ ]:
fig = plt.figure()
gs = fig.add_gridspec(2, 2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 0])
ax4 = fig.add_subplot(gs[1, 1])
fig.set_figheight(8)
fig.set_figwidth(12)

axes = [ax1, ax2, ax3, ax4]
subplot_params = [(.17, .65, .1, .1), (.60, .65, .1, .1), (.17, .14, .1, .1), (.60, .14, .1, .1)]
titles = ['a', 'b', 'c', 'd']

for i, ax in enumerate(axes):
    omegas, detunings = controls_array[i]
    time = timings_array[i]
    max_control = max_control_array[i]
    
    ax.plot(time, omegas, label = 'Ω')
    ax.plot(time, detunings, label = 'Δ')
    ax.set_ylabel(r"Ω,Δ ($\xi$)", rotation = 90)
    ax.set_xlabel(r"t ($\xi^{-1}$)")
    ax.set_ylim((-(max_control + 0.1), (max_control + 0.1)))
    ax.set_title('(' + titles[i] + ')', loc = "right", fontsize = 10)
    ax.legend()

    l, b, h, w = subplot_params[i]
    fidelities = fidelities_array[i]
    ax5 = fig.add_axes([l, b, w, h])
    ax5.plot(time, fidelities)
    ax5.set_ylabel("Fidelity", rotation = 90)

In [ ]:
final_states_array

In [ ]:
[torch.abs(state)**2 for state in final_states_array]

In [ ]:
fidelities = fidelities_array[0]
population1, population2, population3 = populations_array[0]
losses = timing_losses[0]
omegas, detunings = controls_array[0]
time = timings_array[0]

fig = plt.figure()
gs = fig.add_gridspec(2, 2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[1, :-1])
fig.set_figheight(8)
fig.set_figwidth(12)

ax1.plot(time, omegas, label = 'Ω')
ax1.plot(time, detunings, label = 'Δ')
ax1.set_ylabel(r"Ω,Δ ($\xi$)", rotation = 90)
ax1.set_xlabel(r"t ($\xi^{-1}$)")
# ax1.set_ylim((-1.1, 1.1))
ax1.legend()
ax1.set_title('(a)', loc = "right", fontsize = 10)

ax2.plot(time, fidelities)
ax2.axhline(y = 0.9999, color = 'r', linestyle = '--', label = '0.9999')
ax2.set_ylabel("Fidelity", rotation = 90, fontsize = 12)
ax2.set_xlabel(r"t ($\xi^{-1}$)")
ax2.legend(loc = 'lower right')
ax1.set_title('(b)', loc = "right", fontsize = 10)

ax4.plot(time, population1, label = r"$P_1$")
ax4.plot(time, population2, label = r"$P_2$")
ax4.plot(time, population3, label = r"$P_3$")
ax4.set_ylabel("Populations", rotation = 90, fontsize = 12)
ax4.set_xlabel(r"t ($\xi^{-1}$)")
ax4.legend()
ax4.set_title('(c)', loc = "right", fontsize = 10)

epochs_series = np.arange(0, len(losses), 1, dtype = np.int32)
ax3.plot(epochs_series, losses, linewidth=2.0)
ax3.set_ylabel("Loss")
ax3.set_xlabel("epochs")
ax3.set_yscale("log")
ax3.set_title('(d)', loc = "right", fontsize = 10)